In [20]:
import sys
sys.path.append("../src")
import os
import pandas as pd 
import numpy as np
import json
import glob
import shutil
import matplotlib.pyplot as plt
import PIL
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import xception

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2
import tensorflow as tf
from tensorflow import keras
from model_trans import *

from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import ResNet50V2

#from tensorflow.keras.applications.xception import preprocess_input


In [34]:
def build_model_v2(num_classes, size = 224):
    
    model = Sequential()
    
    tr = ResNet50V2(include_top = False,
                       weights = "imagenet",
                       input_shape = (size, size, 3))
    
    #Transfer Learning
    for layer in tr.layers[:14]:
        layer.trainable = False
        model.add(layer)
    
    #Convolution layers
    model.add(Conv2D(32, (3, 3), padding = "same"))
    model.add(Conv2D(32, (3, 3), padding = "same"))
    model.add(BatchNormalization(axis = -1))
    model.add(layers.Activation(activations.relu))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    

    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(BatchNormalization(axis = -1))
    model.add(layers.Activation(activations.relu))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(Conv2D(64, (3, 3), padding = "same"))
    model.add(BatchNormalization(axis = -1))
    model.add(layers.Activation(activations.relu))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(Conv2D(128, (3, 3), padding = "same"))
    model.add(BatchNormalization(axis = -1))
    model.add(layers.Activation(activations.relu))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    #Dense layers
    model.add(Flatten())
    model.add(Dense(512, activation = "relu"))
    model.add(Dropout(0.7))
    model.add(BatchNormalization(axis = -1))
    model.add(Dense(256, activation = "relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(Dense(128, activation = "relu"))
    model.add(BatchNormalization(axis = -1))
    model.add(Dense(num_classes, activation = "softmax"))
    
    return model

In [3]:
os.path.abspath(os.curdir)

'/home/t6hung/DSC180/DSC180_capstone/Racial_Classification_XAI_Model/notebook'

In [6]:
json_path = "../config/parameters.json"

with open(json_path) as param:
    data = json.load(param)
    model_param = data["model_param"]
    data_info = data["load_data"]
    integrated = data["integrated_grad"]
param.close()

lr, epochs, batch_size, mapping_path, save_path, log_path = model_param.values()
train_label_path, train_image_path, valid_label_path, valid_image_path, target, size = data_info.values()

In [5]:
img_name = "{}.jpg".format(1)
sample_path = os.path.join(valid_image_path, img_name)
sample_label_df = pd.read_csv(valid_label_path)
sample_label = sample_label_df[sample_label_df["file"].str.contains(img_name)]["race"].values[0]
    

In [15]:
mod = build_model(7)

In [16]:
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 230, 230, 3)       0         
_________________________________________________________________
conv1_conv (Conv2D)          (None, 112, 112, 64)      9472      
_________________________________________________________________
pool1_pad (ZeroPadding2D)    (None, 114, 114, 64)      0         
_________________________________________________________________
pool1_pool (MaxPooling2D)    (None, 56, 56, 64)        0         
_________________________________________________________________
conv2_block1_preact_bn (Batc (None, 56, 56, 64)        256       
_________________________________________________________________
conv2_block1_preact_relu (Ac (None, 56, 56, 64)        0         
_________________________________________________________________
conv2_block1_1_conv (Conv2D) (None, 56, 56, 64)        4

In [35]:
mod_2 = build_model_v2(7)

In [36]:
mod_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 230, 230, 3)       0         
_________________________________________________________________
conv1_conv (Conv2D)          (None, 112, 112, 64)      9472      
_________________________________________________________________
pool1_pad (ZeroPadding2D)    (None, 114, 114, 64)      0         
_________________________________________________________________
pool1_pool (MaxPooling2D)    (None, 56, 56, 64)        0         
_________________________________________________________________
conv2_block1_preact_bn (Batc (None, 56, 56, 64)        256       
_________________________________________________________________
conv2_block1_preact_relu (Ac (None, 56, 56, 64)        0         
_________________________________________________________________
conv2_block1_1_conv (Conv2D) (None, 56, 56, 64)       